In [1]:
import torch

In [2]:
print(torch.__version__)

2.2.0+cu121


In [3]:
import json
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import random
from skimage import io, transform, util

In [4]:
import torch.nn as nn
import torchvision.transforms.functional as TF
from torchvision import models
import math
import time
import warnings; warnings.simplefilter('ignore')
import matplotlib.pyplot as plt
import os
from torchvision.models import resnet18

In [5]:
class camera_image_network(torch.nn.Module):
    """
    output size is 512 x 512 x 4
    """
    def __init__(self, orig_resnet, n_classes=9):
        super().__init__()
        self.orig_resnet = orig_resnet
        self.orig_resnet.avgpool = nn.AvgPool2d(kernel_size=(8,8), stride=(4,4))
        self.orig_resnet.fc = nn.Linear(in_features=3*3*512, out_features=1000)
        self.orig_resnet.out = nn.Linear(in_features=1000, out_features=n_classes)

    def forward(self, x):
        # encoder
        x1 = self.orig_resnet.conv1(x) # 64 x 256 x 256
        x2 = self.orig_resnet.bn1(x1) 
        x3 = self.orig_resnet.relu(x2)
        x4 = self.orig_resnet.maxpool(x3) # 64 x 128 x 128
        x5 = self.orig_resnet.layer1(x4) # 64 x 128 x 128
        x6 = self.orig_resnet.layer2(x5) # 128 x 64 x 64
        x7 = self.orig_resnet.layer3(x6) # 256 x 32 x 32
        x8 = self.orig_resnet.layer4(x7) # 512 x 16 x 16
        x9 = self.orig_resnet.avgpool(x8) # 512 x 3 x 3
        x10 = x9.reshape(x9.shape[0], -1) # N x (512 x 3 x 3)
        x11 = self.orig_resnet.relu(self.orig_resnet.fc(x10)) # N x 1000
        x12 = self.orig_resnet.out(x11) # N x 9
        return x12

In [6]:
# Camera network
resnet_camera = models.resnet18(pretrained=False)
# choose resnet of your choice
n_input_channel_camera = 4
resnet_camera.conv1 = torch.nn.Conv2d(n_input_channel_camera, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device =", device)
resnet_enc_dec_concat_more = camera_image_network(resnet_camera) 
model_test_camera = resnet_enc_dec_concat_more.to(device)
    
# load previous model
checkpoint_camera = torch.load("../dataset_for_all/navigation_network/best_model/epoch_178val_checkpoint.pth.tar")
model_test_camera.load_state_dict(checkpoint_camera['state_dict'])
model_test_camera = model_test_camera.to(device).eval()

device = cuda:0


In [7]:
# validation accuracy calculation for 5400 images
sz = 20
error = 0
correct = 0
count = 0
csv_path = "./validation_tool_tip_and_targets_label.csv"
labels = pd.read_csv(csv_path, header = None)
for ii in range(5400):
    label = labels.iloc[ii, 0:2].values
    target = labels.iloc[ii, 2:4].values
    input_image = io.imread("../dataset_for_all/navigation_network/validation_data/camera_image_{:06d}".format(ii) + ".jpg")
    target_image = np.zeros((512, 512))
    target_image[int(target[1])-sz:int(target[1])+sz+1, int(target[0])-sz:int(target[0])+sz+1] = 1 
    target_image = target_image[:,:,np.newaxis]*255 
    image = np.concatenate((input_image, target_image), axis = 2) / 255.0
    print(image.shape)

    image = TF.to_tensor(image) # C x H x W
    image = image.unsqueeze(0).float().to(device) # N x C x H x W

    outputs = model_test_camera(image)
    outputs = outputs.cpu().detach().numpy()
    outputs = outputs[0]
    argmax_output = np.argmax(outputs)
    print("Predicted result = ", argmax_output)
    v = target - label
    angle = np.arctan2(-v[1], v[0]) * 180 / np.pi
    if angle >= 157.5 and angle <= 180 or angle < -157.5:
        cur_class_label = 0
    else:
        cur_class_label = int((angle + 202.5) / 45)
    if (np.linalg.norm(v) < 2.5):
        cur_class_label = 8
    print("cur_class_label = ", cur_class_label)
    abs_error = abs(cur_class_label - argmax_output)
    if (abs_error == 0):
        correct += 1
    count += 1
    if (cur_class_label == 8 and argmax_output != 8):
        error += 4
    elif (abs_error > 4):
        error += 8 - abs_error
    else:
        error += abs_error
    print("No. ", ii)
    print("currenet mean error = ", error / count)
    print("accuracy = ", correct / count)

(512, 512, 4)
Predicted result =  0
cur_class_label =  0
No.  0
currenet mean error =  0.0
accuracy =  1.0
(512, 512, 4)
Predicted result =  7
cur_class_label =  7
No.  1
currenet mean error =  0.0
accuracy =  1.0
(512, 512, 4)
Predicted result =  6
cur_class_label =  6
No.  2
currenet mean error =  0.0
accuracy =  1.0
(512, 512, 4)
Predicted result =  0
cur_class_label =  0
No.  3
currenet mean error =  0.0
accuracy =  1.0
(512, 512, 4)
Predicted result =  1
cur_class_label =  1
No.  4
currenet mean error =  0.0
accuracy =  1.0
(512, 512, 4)
Predicted result =  6
cur_class_label =  6
No.  5
currenet mean error =  0.0
accuracy =  1.0
(512, 512, 4)
Predicted result =  5
cur_class_label =  5
No.  6
currenet mean error =  0.0
accuracy =  1.0
(512, 512, 4)
Predicted result =  5
cur_class_label =  5
No.  7
currenet mean error =  0.0
accuracy =  1.0
(512, 512, 4)
Predicted result =  4
cur_class_label =  4
No.  8
currenet mean error =  0.0
accuracy =  1.0
(512, 512, 4)
Predicted result =  5
c

(512, 512, 4)
Predicted result =  0
cur_class_label =  0
No.  73
currenet mean error =  0.02702702702702703
accuracy =  0.9594594594594594
(512, 512, 4)
Predicted result =  7
cur_class_label =  7
No.  74
currenet mean error =  0.02666666666666667
accuracy =  0.96
(512, 512, 4)
Predicted result =  4
cur_class_label =  4
No.  75
currenet mean error =  0.02631578947368421
accuracy =  0.9605263157894737
(512, 512, 4)
Predicted result =  0
cur_class_label =  0
No.  76
currenet mean error =  0.025974025974025976
accuracy =  0.961038961038961
(512, 512, 4)
Predicted result =  2
cur_class_label =  2
No.  77
currenet mean error =  0.02564102564102564
accuracy =  0.9615384615384616
(512, 512, 4)
Predicted result =  0
cur_class_label =  0
No.  78
currenet mean error =  0.02531645569620253
accuracy =  0.9620253164556962
(512, 512, 4)
Predicted result =  7
cur_class_label =  7
No.  79
currenet mean error =  0.025
accuracy =  0.9625
(512, 512, 4)
Predicted result =  7
cur_class_label =  7
No.  80
cu

Predicted result =  7
cur_class_label =  7
No.  144
currenet mean error =  0.013793103448275862
accuracy =  0.9793103448275862
(512, 512, 4)
Predicted result =  6
cur_class_label =  6
No.  145
currenet mean error =  0.0136986301369863
accuracy =  0.9794520547945206
(512, 512, 4)
Predicted result =  6
cur_class_label =  6
No.  146
currenet mean error =  0.013605442176870748
accuracy =  0.9795918367346939
(512, 512, 4)
Predicted result =  7
cur_class_label =  7
No.  147
currenet mean error =  0.013513513513513514
accuracy =  0.9797297297297297
(512, 512, 4)
Predicted result =  0
cur_class_label =  0
No.  148
currenet mean error =  0.013422818791946308
accuracy =  0.9798657718120806
(512, 512, 4)
Predicted result =  7
cur_class_label =  7
No.  149
currenet mean error =  0.013333333333333334
accuracy =  0.98
(512, 512, 4)
Predicted result =  4
cur_class_label =  4
No.  150
currenet mean error =  0.013245033112582781
accuracy =  0.9801324503311258
(512, 512, 4)
Predicted result =  0
cur_cla

(512, 512, 4)
Predicted result =  5
cur_class_label =  5
No.  216
currenet mean error =  0.013824884792626729
accuracy =  0.9815668202764977
(512, 512, 4)
Predicted result =  0
cur_class_label =  0
No.  217
currenet mean error =  0.013761467889908258
accuracy =  0.981651376146789
(512, 512, 4)
Predicted result =  2
cur_class_label =  2
No.  218
currenet mean error =  0.0136986301369863
accuracy =  0.9817351598173516
(512, 512, 4)
Predicted result =  5
cur_class_label =  5
No.  219
currenet mean error =  0.013636363636363636
accuracy =  0.9818181818181818
(512, 512, 4)
Predicted result =  1
cur_class_label =  1
No.  220
currenet mean error =  0.013574660633484163
accuracy =  0.9819004524886877
(512, 512, 4)
Predicted result =  4
cur_class_label =  4
No.  221
currenet mean error =  0.013513513513513514
accuracy =  0.9819819819819819
(512, 512, 4)
Predicted result =  0
cur_class_label =  0
No.  222
currenet mean error =  0.013452914798206279
accuracy =  0.9820627802690582
(512, 512, 4)
Pr

(512, 512, 4)
Predicted result =  0
cur_class_label =  0
No.  284
currenet mean error =  0.017543859649122806
accuracy =  0.9789473684210527
(512, 512, 4)
Predicted result =  0
cur_class_label =  0
No.  285
currenet mean error =  0.017482517482517484
accuracy =  0.9790209790209791
(512, 512, 4)
Predicted result =  7
cur_class_label =  7
No.  286
currenet mean error =  0.017421602787456445
accuracy =  0.9790940766550522
(512, 512, 4)
Predicted result =  0
cur_class_label =  0
No.  287
currenet mean error =  0.017361111111111112
accuracy =  0.9791666666666666
(512, 512, 4)
Predicted result =  7
cur_class_label =  7
No.  288
currenet mean error =  0.01730103806228374
accuracy =  0.9792387543252595
(512, 512, 4)
Predicted result =  3
cur_class_label =  3
No.  289
currenet mean error =  0.017241379310344827
accuracy =  0.9793103448275862
(512, 512, 4)
Predicted result =  6
cur_class_label =  6
No.  290
currenet mean error =  0.01718213058419244
accuracy =  0.979381443298969
(512, 512, 4)
Pr

...

Predicted result =  8
cur_class_label =  8
No.  5107
currenet mean error =  0.08496476115896633
accuracy =  0.9461628817541112
(512, 512, 4)
Predicted result =  8
cur_class_label =  8
No.  5108
currenet mean error =  0.08494813074965747
accuracy =  0.9461734194558622
(512, 512, 4)
Predicted result =  0
cur_class_label =  0
No.  5109
currenet mean error =  0.08493150684931507
accuracy =  0.9461839530332681
(512, 512, 4)
Predicted result =  2
cur_class_label =  8
No.  5110
currenet mean error =  0.08569751516337312
accuracy =  0.9459988260614361
(512, 512, 4)
Predicted result =  8
cur_class_label =  8
No.  5111
currenet mean error =  0.08568075117370892
accuracy =  0.9460093896713615
(512, 512, 4)
Predicted result =  8
cur_class_label =  8
No.  5112
currenet mean error =  0.08566399374144337
accuracy =  0.9460199491492275
(512, 512, 4)
Predicted result =  8
cur_class_label =  8
No.  5113
currenet mean error =  0.08564724286272976
accuracy =  0.946030504497458
(512, 512, 4)
Predicted resu

(512, 512, 4)
Predicted result =  8
cur_class_label =  8
No.  5166
currenet mean error =  0.08612347590478034
accuracy =  0.9460034836462163
(512, 512, 4)
Predicted result =  0
cur_class_label =  0
No.  5167
currenet mean error =  0.08610681114551083
accuracy =  0.9460139318885449
(512, 512, 4)
Predicted result =  8
cur_class_label =  8
No.  5168
currenet mean error =  0.0860901528342039
accuracy =  0.9460243760882182
(512, 512, 4)
Predicted result =  8
cur_class_label =  8
No.  5169
currenet mean error =  0.086073500967118
accuracy =  0.9460348162475822
(512, 512, 4)
Predicted result =  8
cur_class_label =  4
No.  5170
currenet mean error =  0.08683040030941791
accuracy =  0.945851866176755
(512, 512, 4)
Predicted result =  4
cur_class_label =  4
No.  5171
currenet mean error =  0.08681361175560712
accuracy =  0.945862335653519
(512, 512, 4)
Predicted result =  4
cur_class_label =  4
No.  5172
currenet mean error =  0.08679682969263483
accuracy =  0.945872801082544
(512, 512, 4)
Predi

Predicted result =  2
cur_class_label =  2
No.  5235
currenet mean error =  0.08976317799847211
accuracy =  0.9446142093200917
(512, 512, 4)
Predicted result =  8
cur_class_label =  8
No.  5236
currenet mean error =  0.08974603780790529
accuracy =  0.9446247851823563
(512, 512, 4)
Predicted result =  1
cur_class_label =  1
No.  5237
currenet mean error =  0.08972890416189386
accuracy =  0.944635357006491
(512, 512, 4)
Predicted result =  8
cur_class_label =  8
No.  5238
currenet mean error =  0.08971177705669021
accuracy =  0.9446459247948081
(512, 512, 4)
Predicted result =  8
cur_class_label =  5
No.  5239
currenet mean error =  0.09026717557251908
accuracy =  0.9444656488549619
(512, 512, 4)
Predicted result =  4
cur_class_label =  4
No.  5240
currenet mean error =  0.09024995229917955
accuracy =  0.9444762449914138
(512, 512, 4)
Predicted result =  4
cur_class_label =  4
No.  5241
currenet mean error =  0.09023273559710035
accuracy =  0.9444868370850821
(512, 512, 4)
Predicted resu

(512, 512, 4)
Predicted result =  0
cur_class_label =  0
No.  5294
currenet mean error =  0.08970727101038715
accuracy =  0.9446647780925401
(512, 512, 4)
Predicted result =  4
cur_class_label =  8
No.  5295
currenet mean error =  0.09044561933534744
accuracy =  0.9444864048338368
(512, 512, 4)
Predicted result =  0
cur_class_label =  0
No.  5296
currenet mean error =  0.0904285444591278
accuracy =  0.9444968850292619
(512, 512, 4)
Predicted result =  8
cur_class_label =  8
No.  5297
currenet mean error =  0.09041147602869007
accuracy =  0.9445073612684032
(512, 512, 4)
Predicted result =  8
cur_class_label =  8
No.  5298
currenet mean error =  0.09039441404038498
accuracy =  0.9445178335535006
(512, 512, 4)
Predicted result =  8
cur_class_label =  8
No.  5299
currenet mean error =  0.09037735849056604
accuracy =  0.9445283018867925
(512, 512, 4)
Predicted result =  8
cur_class_label =  8
No.  5300
currenet mean error =  0.09036030937558952
accuracy =  0.944538766270515
(512, 512, 4)
P

(512, 512, 4)
Predicted result =  8
cur_class_label =  0
No.  5364
currenet mean error =  0.09133271202236719
accuracy =  0.9438956197576888
(512, 512, 4)
Predicted result =  8
cur_class_label =  8
No.  5365
currenet mean error =  0.09131569139023481
accuracy =  0.9439060752888557
(512, 512, 4)
Predicted result =  6
cur_class_label =  8
No.  5366
currenet mean error =  0.09204397242407304
accuracy =  0.9437302030929756
(512, 512, 4)
Predicted result =  8
cur_class_label =  8
No.  5367
currenet mean error =  0.092026825633383
accuracy =  0.9437406855439643
(512, 512, 4)
Predicted result =  8
cur_class_label =  8
No.  5368
currenet mean error =  0.09200968523002422
accuracy =  0.9437511640901471
(512, 512, 4)
Predicted result =  4
cur_class_label =  4
No.  5369
currenet mean error =  0.0919925512104283
accuracy =  0.9437616387337058
(512, 512, 4)
Predicted result =  8
cur_class_label =  8
No.  5370
currenet mean error =  0.0919754235710296
accuracy =  0.94377210947682
(512, 512, 4)
Predi

In [11]:
# test accuracy calculation for 300 x 10 = 3000 targets, since we randomly generated targets, test accuracy varies each time
sz = 20
error = 0
correct = 0
count = 0
for i in range(9):
    for ii in range(300):
        base_path = "test_label"
        json_path = os.path.join(base_path, "camera_image_{:06d}".format(ii) + ".jpg.json")
        with open(json_path) as json_file:
            data = json.load(json_file)
            label = data['objects'][0]['points']['exterior'][0]
            label = np.array(label)
            if i == -1:
                target = label
            else:
                x = random.randint(-50, 51) + label[0]
                y = random.randint(-50, 51) + label[1]
                if (x > 511):
                    x = 511
                elif (x < 0):
                    x = 0
                if (y > 511):
                    y = 511
                elif (y < 0):
                    y = 0
                target = [x, y]
                target = np.array(target)
        input_image = io.imread("../dataset_for_all/navigation_network/test_data/camera_image_{:06d}".format(ii) + ".jpg")
        target_image = np.zeros((512, 512))
        target_image[int(target[1])-sz:int(target[1])+sz+1, int(target[0])-sz:int(target[0])+sz+1] = 1 
        target_image = target_image[:,:,np.newaxis]*255 #
        image = np.concatenate((input_image, target_image), axis = 2) / 255.0
        image = TF.to_tensor(image) # C x H x W
        image = image.unsqueeze(0).float().to(device) # N x C x H x W

        outputs = model_test_camera(image)
        outputs = outputs.cpu().detach().numpy()
        outputs = outputs[0]
        argmax_output = np.argmax(outputs)
        print("Predicted result = ", argmax_output)
        v = target - label
        angle = np.arctan2(-v[1], v[0]) * 180 / np.pi
        if angle >= 157.5 and angle <= 180 or angle < -157.5:
            cur_class_label = 0
        else:
            cur_class_label = int((angle + 202.5) / 45)
        if (np.linalg.norm(v) < 2.5):
            cur_class_label = 8
        print("cur_class_label = ", cur_class_label)
        abs_error = abs(cur_class_label - argmax_output)
        if (abs_error == 0):
            correct += 1
        count += 1
        if (cur_class_label == 8 and argmax_output != 8):
            error += 4
        elif (abs_error > 4):
            error += 8 - abs_error
        else:
            error += abs_error
        print("No. ", i * 300 + ii)
        print("currenet mean error = ", error / count)
        print("accuracy = ", correct / count)
for i in range(1):
    for ii in range(300):
        base_path = "test_label"
        json_path = os.path.join(base_path, "camera_image_{:06d}".format(ii) + ".jpg.json")
        with open(json_path) as json_file:
            data = json.load(json_file)
            label = data['objects'][0]['points']['exterior'][0]
            label = np.array(label)
            if i == -1:
                target = label
            else:
                x = label[0]
                y = label[1]
                if (x > 511):
                    x = 511
                elif (x < 0):
                    x = 0
                if (y > 511):
                    y = 511
                elif (y < 0):
                    y = 0
                target = [x, y]
                target = np.array(target)
        input_image = io.imread("../dataset_for_all/navigation_network/test_data/camera_image_{:06d}".format(ii) + ".jpg")
        target_image = np.zeros((512, 512))
        target_image[int(target[1])-sz:int(target[1])+sz+1, int(target[0])-sz:int(target[0])+sz+1] = 1 
        target_image = target_image[:,:,np.newaxis]*255
        image = np.concatenate((input_image, target_image), axis = 2) / 255.0
        image = TF.to_tensor(image) # C x H x W
        image = image.unsqueeze(0).float().to(device) # N x C x H x W

        outputs = model_test_camera(image)
        outputs = outputs.cpu().detach().numpy()
        outputs = outputs[0]
        argmax_output = np.argmax(outputs)
        print("Predicted result = ", argmax_output)
        v = target - label
        angle = np.arctan2(-v[1], v[0]) * 180 / np.pi
        if angle >= 157.5 and angle <= 180 or angle < -157.5:
            cur_class_label = 0
        else:
            cur_class_label = int((angle + 202.5) / 45)
        if (np.linalg.norm(v) < 2.5):
            cur_class_label = 8
        print("cur_class_label = ", cur_class_label)
        abs_error = abs(cur_class_label - argmax_output)
        if (abs_error == 0):
            correct += 1
        count += 1
        if (cur_class_label == 8 and argmax_output != 8):
            error += 4
        elif (abs_error > 4):
            error += 8 - abs_error
        else:
            error += abs_error
        print("No. ", i * 300 + ii + 2700)
        print("currenet mean error = ", error / count)
        print("accuracy = ", correct / count)

Predicted result =  7
cur_class_label =  7
No.  0
currenet mean error =  0.0
accuracy =  1.0
Predicted result =  0
cur_class_label =  0
No.  1
currenet mean error =  0.0
accuracy =  1.0
Predicted result =  1
cur_class_label =  1
No.  2
currenet mean error =  0.0
accuracy =  1.0
Predicted result =  1
cur_class_label =  1
No.  3
currenet mean error =  0.0
accuracy =  1.0
Predicted result =  2
cur_class_label =  2
No.  4
currenet mean error =  0.0
accuracy =  1.0
Predicted result =  3
cur_class_label =  3
No.  5
currenet mean error =  0.0
accuracy =  1.0
Predicted result =  6
cur_class_label =  6
No.  6
currenet mean error =  0.0
accuracy =  1.0
Predicted result =  0
cur_class_label =  0
No.  7
currenet mean error =  0.0
accuracy =  1.0
Predicted result =  5
cur_class_label =  5
No.  8
currenet mean error =  0.0
accuracy =  1.0
Predicted result =  4
cur_class_label =  4
No.  9
currenet mean error =  0.0
accuracy =  1.0
Predicted result =  1
cur_class_label =  1
No.  10
currenet mean error

Predicted result =  4
cur_class_label =  4
No.  88
currenet mean error =  0.033707865168539325
accuracy =  0.9662921348314607
Predicted result =  3
cur_class_label =  3
No.  89
currenet mean error =  0.03333333333333333
accuracy =  0.9666666666666667
Predicted result =  3
cur_class_label =  3
No.  90
currenet mean error =  0.03296703296703297
accuracy =  0.967032967032967
Predicted result =  6
cur_class_label =  6
No.  91
currenet mean error =  0.03260869565217391
accuracy =  0.967391304347826
Predicted result =  2
cur_class_label =  2
No.  92
currenet mean error =  0.03225806451612903
accuracy =  0.967741935483871
Predicted result =  0
cur_class_label =  0
No.  93
currenet mean error =  0.031914893617021274
accuracy =  0.9680851063829787
Predicted result =  5
cur_class_label =  5
No.  94
currenet mean error =  0.031578947368421054
accuracy =  0.968421052631579
Predicted result =  5
cur_class_label =  5
No.  95
currenet mean error =  0.03125
accuracy =  0.96875
Predicted result =  2
cu

Predicted result =  3
cur_class_label =  3
No.  160
currenet mean error =  0.06832298136645963
accuracy =  0.9503105590062112
Predicted result =  2
cur_class_label =  2
No.  161
currenet mean error =  0.06790123456790123
accuracy =  0.9506172839506173
Predicted result =  6
cur_class_label =  6
No.  162
currenet mean error =  0.06748466257668712
accuracy =  0.950920245398773
Predicted result =  1
cur_class_label =  1
No.  163
currenet mean error =  0.06707317073170732
accuracy =  0.9512195121951219
Predicted result =  5
cur_class_label =  5
No.  164
currenet mean error =  0.06666666666666667
accuracy =  0.9515151515151515
Predicted result =  3
cur_class_label =  3
No.  165
currenet mean error =  0.06626506024096386
accuracy =  0.9518072289156626
Predicted result =  3
cur_class_label =  3
No.  166
currenet mean error =  0.0658682634730539
accuracy =  0.9520958083832335
Predicted result =  4
cur_class_label =  4
No.  167
currenet mean error =  0.06547619047619048
accuracy =  0.95238095238

Predicted result =  1
cur_class_label =  1
No.  235
currenet mean error =  0.059322033898305086
accuracy =  0.9533898305084746
Predicted result =  1
cur_class_label =  1
No.  236
currenet mean error =  0.05907172995780591
accuracy =  0.9535864978902954
Predicted result =  5
cur_class_label =  5
No.  237
currenet mean error =  0.058823529411764705
accuracy =  0.9537815126050421
Predicted result =  7
cur_class_label =  7
No.  238
currenet mean error =  0.058577405857740586
accuracy =  0.9539748953974896
Predicted result =  2
cur_class_label =  2
No.  239
currenet mean error =  0.058333333333333334
accuracy =  0.9541666666666667
Predicted result =  3
cur_class_label =  3
No.  240
currenet mean error =  0.058091286307053944
accuracy =  0.9543568464730291
Predicted result =  2
cur_class_label =  2
No.  241
currenet mean error =  0.05785123966942149
accuracy =  0.9545454545454546
Predicted result =  3
cur_class_label =  2
No.  242
currenet mean error =  0.06172839506172839
accuracy =  0.9506

Predicted result =  5
cur_class_label =  5
No.  310
currenet mean error =  0.05787781350482315
accuracy =  0.9517684887459807
Predicted result =  3
cur_class_label =  3
No.  311
currenet mean error =  0.057692307692307696
accuracy =  0.9519230769230769
Predicted result =  2
cur_class_label =  2
No.  312
currenet mean error =  0.05750798722044728
accuracy =  0.952076677316294
Predicted result =  4
cur_class_label =  4
No.  313
currenet mean error =  0.05732484076433121
accuracy =  0.9522292993630573
Predicted result =  5
cur_class_label =  5
No.  314
currenet mean error =  0.05714285714285714
accuracy =  0.9523809523809523
Predicted result =  1
cur_class_label =  1
No.  315
currenet mean error =  0.056962025316455694
accuracy =  0.9525316455696202
Predicted result =  0
cur_class_label =  0
No.  316
currenet mean error =  0.056782334384858045
accuracy =  0.9526813880126183
Predicted result =  2
cur_class_label =  2
No.  317
currenet mean error =  0.05660377358490566
accuracy =  0.9528301

...

Predicted result =  8
cur_class_label =  8
No.  2701
currenet mean error =  0.05477424130273871
accuracy =  0.9463360473723168
Predicted result =  8
cur_class_label =  8
No.  2702
currenet mean error =  0.05475397706252312
accuracy =  0.9463559008509064
Predicted result =  8
cur_class_label =  8
No.  2703
currenet mean error =  0.05473372781065089
accuracy =  0.9463757396449705
Predicted result =  8
cur_class_label =  8
No.  2704
currenet mean error =  0.05471349353049908
accuracy =  0.9463955637707948
Predicted result =  8
cur_class_label =  8
No.  2705
currenet mean error =  0.05469327420546933
accuracy =  0.9464153732446415
Predicted result =  8
cur_class_label =  8
No.  2706
currenet mean error =  0.05467306981898781
accuracy =  0.9464351680827484
Predicted result =  8
cur_class_label =  8
No.  2707
currenet mean error =  0.05465288035450517
accuracy =  0.9464549483013294
Predicted result =  8
cur_class_label =  8
No.  2708
currenet mean error =  0.05463270579549649
accuracy =  0.9

Predicted result =  8
cur_class_label =  8
No.  2769
currenet mean error =  0.05631768953068592
accuracy =  0.9469314079422383
Predicted result =  8
cur_class_label =  8
No.  2770
currenet mean error =  0.05629736557199567
accuracy =  0.9469505593648503
Predicted result =  8
cur_class_label =  8
No.  2771
currenet mean error =  0.05627705627705628
accuracy =  0.946969696969697
Predicted result =  8
cur_class_label =  8
No.  2772
currenet mean error =  0.05625676163000361
accuracy =  0.9469888207717274
Predicted result =  3
cur_class_label =  8
No.  2773
currenet mean error =  0.05767844268204758
accuracy =  0.946647440519106
Predicted result =  8
cur_class_label =  8
No.  2774
currenet mean error =  0.05765765765765766
accuracy =  0.9466666666666667
Predicted result =  8
cur_class_label =  8
No.  2775
currenet mean error =  0.05763688760806916
accuracy =  0.946685878962536
Predicted result =  8
cur_class_label =  8
No.  2776
currenet mean error =  0.05761613251710479
accuracy =  0.9467

Predicted result =  8
cur_class_label =  8
No.  2842
currenet mean error =  0.06331340133661625
accuracy =  0.9461836088638762
Predicted result =  8
cur_class_label =  8
No.  2843
currenet mean error =  0.06329113924050633
accuracy =  0.9462025316455697
Predicted result =  8
cur_class_label =  8
No.  2844
currenet mean error =  0.0632688927943761
accuracy =  0.9462214411247803
Predicted result =  8
cur_class_label =  8
No.  2845
currenet mean error =  0.06324666198172874
accuracy =  0.9462403373155306
Predicted result =  8
cur_class_label =  8
No.  2846
currenet mean error =  0.06322444678609063
accuracy =  0.946259220231823
Predicted result =  6
cur_class_label =  8
No.  2847
currenet mean error =  0.06460674157303371
accuracy =  0.9459269662921348
Predicted result =  8
cur_class_label =  8
No.  2848
currenet mean error =  0.06458406458406458
accuracy =  0.9459459459459459
Predicted result =  8
cur_class_label =  8
No.  2849
currenet mean error =  0.06456140350877193
accuracy =  0.945

Predicted result =  8
cur_class_label =  8
No.  2919
currenet mean error =  0.08767123287671233
accuracy =  0.9410958904109589
Predicted result =  8
cur_class_label =  8
No.  2920
currenet mean error =  0.08764121876069839
accuracy =  0.9411160561451558
Predicted result =  8
cur_class_label =  8
No.  2921
currenet mean error =  0.08761122518822724
accuracy =  0.9411362080766599
Predicted result =  8
cur_class_label =  8
No.  2922
currenet mean error =  0.08758125213821416
accuracy =  0.9411563462196374
Predicted result =  8
cur_class_label =  8
No.  2923
currenet mean error =  0.08755129958960328
accuracy =  0.9411764705882353
Predicted result =  8
cur_class_label =  8
No.  2924
currenet mean error =  0.08752136752136752
accuracy =  0.9411965811965812
Predicted result =  8
cur_class_label =  8
No.  2925
currenet mean error =  0.08749145591250855
accuracy =  0.9412166780587833
Predicted result =  8
cur_class_label =  8
No.  2926
currenet mean error =  0.08746156474205671
accuracy =  0.9

Predicted result =  8
cur_class_label =  8
No.  2992
currenet mean error =  0.09355162044771133
accuracy =  0.9405278984296692
Predicted result =  8
cur_class_label =  8
No.  2993
currenet mean error =  0.09352037408149633
accuracy =  0.9405477621910487
Predicted result =  8
cur_class_label =  8
No.  2994
currenet mean error =  0.09348914858096828
accuracy =  0.940567612687813
Predicted result =  8
cur_class_label =  8
No.  2995
currenet mean error =  0.09345794392523364
accuracy =  0.9405874499332443
Predicted result =  8
cur_class_label =  8
No.  2996
currenet mean error =  0.09342676009342676
accuracy =  0.9406072739406073
Predicted result =  8
cur_class_label =  8
No.  2997
currenet mean error =  0.0933955970647098
accuracy =  0.9406270847231488
Predicted result =  4
cur_class_label =  8
No.  2998
currenet mean error =  0.09469823274424809
accuracy =  0.9403134378126042
Predicted result =  3
cur_class_label =  8
No.  2999
currenet mean error =  0.096
accuracy =  0.94
